In [1]:
import os

os.chdir("..")
print(f"Changed working directory to: {os.getcwd()}")

Changed working directory to: /home/jovyan/work/FlareSense


In [2]:
import os

os.getcwd()

'/home/jovyan/work/FlareSense'

In [3]:
import torch
import mlflow
import dagshub
import torchmetrics
import src.utils.data as data
import pytorch_lightning as pl

from torchvision import transforms
from tqdm.notebook import tqdm
from src.models.ViTB16BinaryClassifier import ViTB16BinaryClassifier

mlflow.pytorch.autolog()
torch.set_float32_matmul_precision("high")

/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_config.py:318: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)
2023/11/22 16:19:13 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of pytorch. If you encounter errors during autologging, try upgrading / downgrading pytorch to a supported version, or try upgrading MLflow.
2023-11-22 16:19:13.239480: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI 

## Training

Defining the data and model

In [4]:
model = ViTB16BinaryClassifier(lr=1e-3, weight_decay=1e-4)
data_folder_path = "data/raw/burst_images/"

data_module = data.ECallistoDataModule(
    data_folder=data_folder_path,
    transform=transforms.Compose(
        [
            transforms.ToPILImage(),
            transforms.Resize((224, 224), antialias=True), # ViT setzt 224 voraus, bisher 193x240
            transforms.ToTensor(),
        ]
    ),
    batch_size=64,
    num_workers=10,
    val_ratio=0.15,
    test_ratio=0.15,
    split_by_date=True,
    noburst_to_burst_ratio=4,
    filter_instruments=["australia_assa_02"],
)
data_module.setup()

Not enough validation bursts (40), minimum 119 needed
Not enough test bursts (101), minimum 119 needed
Reshuffling...

Not enough validation bursts (99), minimum 119 needed
Not enough test bursts (71), minimum 119 needed
Reshuffling...

Not enough test bursts (109), minimum 119 needed
Too many validation bursts (401), maximum 266 allowed
Reshuffling...

Not enough test bursts (55), minimum 119 needed
Too many validation bursts (338), maximum 266 allowed
Reshuffling...

Not enough validation bursts (117), minimum 119 needed
Reshuffling...

Not enough validation bursts (81), minimum 119 needed
Not enough test bursts (101), minimum 119 needed
Reshuffling...

Not enough validation bursts (116), minimum 119 needed
Reshuffling...

Dataset split successfully
Train:		807 bursts
Validation:	169 bursts
Test:		207 bursts


Training and logging the model

In [5]:
dagshub.init("FlareSense", "FlareSense", mlflow=True)
mlflow.start_run()

mlflow.log_params({
    "model": "ViT16",
    "batch_size": data_module.batch_size,               
    "val_ratio": data_module.val_ratio,
    "test_ratio": data_module.test_ratio,
    "min_factor_val_test": data_module.min_factor_val_test,
    "max_factor_val_test": data_module.max_factor_val_test,
    "noburst_to_burst_ratio": data_module.noburst_to_burst_ratio,
    "split_by_date": data_module.split_by_date,
    "filter_instruments": data_module.filter_instruments,
})

run_id = mlflow.active_run().info.run_id
print(f"Run ID: {run_id}")
print(f"Link: https://dagshub.com/FlareSense/FlareSense/experiments/#/experiment/m_{run_id}")

trainer = pl.Trainer(max_epochs=10, log_every_n_steps=1)

trainer.fit(
    model,
    train_dataloaders=data_module.train_dataloader(),
    val_dataloaders=data_module.val_dataloader(),
)

trainer.test(model, dataloaders=data_module.test_dataloader())

mlflow.end_run()

Repository initialized!

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Run ID: 153b02b2825541f4855ae3beef517967
Link: https://dagshub.com/FlareSense/FlareSense/experiments/#/experiment/m_153b02b2825541f4855ae3beef517967


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type              | Params
------------------------------------------------
0 | vit_b_16  | VisionTransformer | 85.8 M
1 | precision | BinaryPrecision   | 0     
2 | recall    | BinaryRecall      | 0     
------------------------------------------------
85.8 M    Trainable params
0         Non-trainable params
85.8 M    Total params
343.198   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
2023/11/22 16:24:27 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/conda/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.19037391245365143    │
│      test_precision       │            0.0            │
│        test_recall        │            0.0            │
└───────────────────────────┴───────────────────────────┘

## Testing

Downloading the model from mlflow and putting it on the correct device

In [6]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

mlflow.set_tracking_uri('https://dagshub.com/FlareSense/Flaresense.mlflow')
model = mlflow.pytorch.load_model(f"runs:/{run_id}/model/").to(device)
model.eval()

ViTB16BinaryClassifier(
  (vit_b_16): VisionTransformer(
    (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (encoder): Encoder(
      (dropout): Dropout(p=0.0, inplace=False)
      (layers): Sequential(
        (encoder_layer_0): EncoderBlock(
          (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (self_attention): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (dropout): Dropout(p=0.0, inplace=False)
          (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): MLPBlock(
            (0): Linear(in_features=768, out_features=3072, bias=True)
            (1): GELU(approximate='none')
            (2): Dropout(p=0.0, inplace=False)
            (3): Linear(in_features=3072, out_features=768, bias=True)
            (4): Dropout(p=0.0, inplace=False)
          )
        )
        (encoder_layer_1): EncoderBlock(
       

Run through testdata

In [7]:
test_labels_list = []
test_preds_list = []
with torch.no_grad():
    for batch in tqdm(data_module.test_dataloader()):
        images, info = batch
        images = images.to(device)
        binary_labels = [0 if label == "no_burst" else 1 for label in info['label']]
        binary_labels = torch.tensor(binary_labels).int().view(-1, 1)
        binary_labels = binary_labels.to(device)
        
        outputs = model(images.expand(-1, 3, -1, -1))
        predictions = (outputs >= 0.5).int()
        
        test_labels_list.append(binary_labels)
        test_preds_list.append(predictions)

  0%|          | 0/68 [00:00<?, ?it/s]

Create confmatrix

In [8]:
test_labels = torch.cat(test_labels_list, dim=0)
test_preds = torch.cat(test_preds_list, dim=0)

confmat_metric = torchmetrics.ConfusionMatrix(num_classes=2, task="binary").to(device)
confmat_metric(test_preds, test_labels)

tensor([[4131,    1],
        [ 207,    0]], device='cuda:0')